<a href="https://colab.research.google.com/github/srinivasbharani/Advanced-Regression---Machine-Learning/blob/main/FGSM_3_Defenses_Version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Normalize pixel values to [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = utils.to_categorical(y_train, num_classes=10)
y_test = utils.to_categorical(y_test, num_classes=10)

In [ ]:
# Create DNN model
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

In [ ]:
# Fast Gradient Sign Method (FGSM) attack
def fgsm_attack(model, x, y, epsilon=0.03):
    x = tf.convert_to_tensor(x)  # Convert input to Tensor if not already
    y = tf.convert_to_tensor(y)  # Convert labels to Tensor if not already

    with tf.GradientTape() as tape:
        tape.watch(x)
        predictions = model(x)
        loss = tf.keras.losses.categorical_crossentropy(y, predictions)
    gradient = tape.gradient(loss, x)
    x_adv = x + epsilon * tf.sign(gradient)
    x_adv = tf.clip_by_value(x_adv, 0, 1)
    return x_adv

In [ ]:
# Adversarial Training
def adversarial_training(model, x_train, y_train, epsilon=0.03, epochs=10, batch_size=32):
    for epoch in range(epochs):
        x_train_adv = fgsm_attack(model, x_train, y_train, epsilon=epsilon)
        model.fit(x_train_adv, y_train, epochs=1, batch_size=batch_size)


In [ ]:
# Helper function for defensive distillation
def softmax_with_temperature(logits, temperature):
    exp_logits = np.exp(logits / temperature)
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)


In [ ]:
# Generate adversarial data
x_test_adv = fgsm_attack(create_model(), x_test, y_test, epsilon=0.03)


In [ ]:
# Create and train the clean model
clean_model = create_model()
clean_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
clean_model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/10
1563/1563 [==============================] - 77s 48ms/step - loss: 1.5519 - accuracy: 0.4336 - val_loss: 1.2947 - val_accuracy: 0.5336
Epoch 2/10
1563/1563 [==============================] - 77s 49ms/step - loss: 1.1916 - accuracy: 0.5779 - val_loss: 1.0806 - val_accuracy: 0.6171
Epoch 3/10
1563/1563 [==============================] - 74s 48ms/step - loss: 1.0397 - accuracy: 0.6322 - val_loss: 1.0382 - val_accuracy: 0.6359
Epoch 4/10
1563/1563 [==============================] - 77s 49ms/step - loss: 0.9376 - accuracy: 0.6720 - val_loss: 0.9516 - val_accuracy: 0.6666
Epoch 5/10
1563/1563 [==============================] - 78s 50ms/step - loss: 0.8584 - accuracy: 0.6972 - val_loss: 0.9005 - val_accuracy: 0.6840
Epoch 6/10
1563/1563 [==============================] - 79s 51ms/step - loss: 0.8009 - accuracy: 0.7184 - val_loss: 0.8995 - val_accuracy: 0.6924
Epoch 7/10
1563/1563 [==============================] - 78s 50ms/step - loss: 0.7545 - accuracy: 0.7365 - val_loss: 0.8595 -

In [ ]:
# Evaluate accuracy on clean test data
_, clean_accuracy = clean_model.evaluate(x_test, y_test, verbose=0)


In [ ]:
# Evaluate accuracy on adversarial test data
_, clean_adv_accuracy = clean_model.evaluate(x_test_adv, y_test, verbose=0)


In [ ]:
# Adversarial Training with FGSM attack on clean model
adv_model = create_model()
adv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
adversarial_training(adv_model, x_train, y_train, epsilon=0.03, epochs=10, batch_size=32)


In [ ]:
# Evaluate accuracy of adversarial training model on clean test data
_, adv_train_accuracy_clean = adv_model.evaluate(x_test, y_test, verbose=0)


In [ ]:
# Evaluate accuracy of adversarial training model on adversarial test data
_, adv_train_accuracy_adv = adv_model.evaluate(x_test_adv, y_test, verbose=0)


In [ ]:
# Adversarial Training with Gradient Masking
adv_model_gm = create_model()
adv_model_gm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
for _ in range(10):
    x_train_masked = x_train + 0.1 * np.sign(np.random.randn(*x_train.shape))
    x_train_masked = np.clip(x_train_masked, 0, 1)
    adv_model_gm.fit(x_train_masked, y_train, epochs=1, batch_size=32)


In [ ]:
# Evaluate accuracy on adversarial training model with gradient masking
_, adv_train_accuracy_gm_clean = adv_model_gm.evaluate(x_test, y_test, verbose=0)


In [ ]:
# Evaluate accuracy on adversarial test data with gradient masking
_, adv_train_accuracy_gm_adv = adv_model_gm.evaluate(x_test_adv, y_test, verbose=0)


In [ ]:
# Defensive Distillation
defensive_distillation_model = create_model()
defensive_distillation_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Soft targets from the trained model with temperature
temperature = 3.0
soft_targets = softmax_with_temperature(clean_model.predict(x_train), temperature)
soft_targets_test = softmax_with_temperature(clean_model.predict(x_test), temperature)

for _ in range(10):
    defensive_distillation_model.fit(x_train, soft_targets, epochs=1, batch_size=32)


In [ ]:
# Evaluate accuracy of defensive distillation model on clean and adversarial test data
_, defensive_distillation_clean_accuracy = defensive_distillation_model.evaluate(x_test, y_test, verbose=0)
x_test_adv_soft = softmax_with_temperature(clean_model.predict(x_test_adv), temperature)
_, defensive_distillation_adv_accuracy = defensive_distillation_model.evaluate(x_test_adv, x_test_adv_soft, verbose=0)


In [ ]:
# Calculate robustness scores
robustness_score_clean = clean_adv_accuracy / clean_accuracy
robustness_score_adv = adv_train_accuracy_adv / adv_train_accuracy_clean
robustness_score_gm_clean = adv_train_accuracy_gm_clean / clean_accuracy
robustness_score_gm_adv = adv_train_accuracy_gm_adv / clean_accuracy
robustness_score_dd_clean = defensive_distillation_clean_accuracy / clean_accuracy
robustness_score_dd_adv = defensive_distillation_adv_accuracy / clean_accuracy


In [ ]:
# Generate confusion matrices
def generate_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()


In [ ]:
# Confusion matrix for clean model on clean test data
y_pred_clean = np.argmax(clean_model.predict(x_test), axis=1)
generate_confusion_matrix(np.argmax(y_test, axis=1), y_pred_clean, "Confusion Matrix (Clean Model, Clean Test Data)")


In [ ]:
# Confusion matrix for clean model on adversarial test data
y_pred_clean_adv = np.argmax(clean_model.predict(x_test_adv), axis=1)
generate_confusion_matrix(np.argmax(y_test, axis=1), y_pred_clean_adv, "Confusion Matrix (Clean Model, Adversarial Test Data)")


In [ ]:
# Confusion matrix for adversarial training model on clean test data
y_pred_adv_train_clean = np.argmax(adv_model.predict(x_test), axis=1)
generate_confusion_matrix(np.argmax(y_test, axis=1), y_pred_adv_train_clean, "Confusion Matrix (Adversarial Training Model, Clean Test Data)")


In [ ]:
# Confusion matrix for adversarial training model on adversarial test data
y_pred_adv_train_adv = np.argmax(adv_model.predict(x_test_adv), axis=1)
generate_confusion_matrix(np.argmax(y_test, axis=1), y_pred_adv_train_adv, "Confusion Matrix (Adversarial Training Model, Adversarial Test Data)")


In [ ]:
# Confusion matrix for adversarial training model with gradient masking on clean test data
y_pred_adv_train_gm_clean = np.argmax(adv_model_gm.predict(x_test), axis=1)
generate_confusion_matrix(np.argmax(y_test, axis=1), y_pred_adv_train_gm_clean, "Confusion Matrix (Adversarial Training Model with Gradient Masking, Clean Test Data)")


In [ ]:
# Confusion matrix for adversarial training model with gradient masking on adversarial test data
y_pred_adv_train_gm_adv = np.argmax(adv_model_gm.predict(x_test_adv), axis=1)
generate_confusion_matrix(np.argmax(y_test, axis=1), y_pred_adv_train_gm_adv, "Confusion Matrix (Adversarial Training Model with Gradient Masking, Adversarial Test Data)")


In [ ]:
# Confusion matrix for defensive distillation model on clean test data
y_pred_def_dist_clean = np.argmax(defensive_distillation_model.predict(x_test), axis=1)
generate_confusion_matrix(np.argmax(y_test, axis=1), y_pred_def_dist_clean, "Confusion Matrix (Defensive Distillation Model, Clean Test Data)")


In [ ]:
# Confusion matrix for defensive distillation model on adversarial test data
y_pred_def_dist_adv = np.argmax(defensive_distillation_model.predict(x_test_adv), axis=1)
generate_confusion_matrix(np.argmax(y_test, axis=1), y_pred_def_dist_adv, "Confusion Matrix (Defensive Distillation Model, Adversarial Test Data)")


In [ ]:
# Print accuracy and robustness scores
print("Clean Model Accuracy:", clean_accuracy)
print("Clean Model Robustness Score:", robustness_score_clean)

print("Adversarial Training Model Accuracy on Clean Data:", adv_train_accuracy_clean)
print("Adversarial Training Model Robustness Score on Clean Data:", robustness_score_adv)

print("Adversarial Training Model with Gradient Masking Accuracy on Clean Data:", adv_train_accuracy_gm_clean)
print("Adversarial Training Model with Gradient Masking Robustness Score on Clean Data:", robustness_score_gm_clean)

print("Defensive Distillation Model Accuracy on Clean Data:", defensive_distillation_clean_accuracy)
print("Defensive Distillation Model Robustness Score on Clean Data:", robustness_score_dd_clean)

In [ ]:
# Plot comparison graph
labels = ['Clean Model', 'Adv Training', 'GM Model', 'Def Distillation']
accuracy_scores = [clean_accuracy, adv_train_accuracy_clean, adv_train_accuracy_gm_clean, defensive_distillation_clean_accuracy]
robustness_scores = [robustness_score_clean, robustness_score_adv, robustness_score_gm_clean, robustness_score_dd_clean]

fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(labels))
width = 0.35

rects1 = ax.bar(x - width/2, accuracy_scores, width, label='Accuracy')
rects2 = ax.bar(x + width/2, robustness_scores, width, label='Robustness')

ax.set_ylabel('Scores')
ax.set_title('Comparison of Accuracy and Robustness')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

fig.tight_layout()
plt.show()